## Statoil Iceberg Classification

### Classify if an image is an iceberg or a ship
### Use Keras/Tensorflow CNN for training

In [2]:
import numpy as np
import pandas as pd
import keras
from matplotlib import pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau
import cv2
np.random.seed(100)

Using TensorFlow backend.


In [3]:
# Read data
train = pd.read_json("train.json")
test = pd.read_json("test.json")

In [4]:
def get_scaled_imgs(df):
    imgs = []
    
    for i, row in df.iterrows():
        #make 75x75 image
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = band_1 + band_2 # plus since log(x*y) = log(x) + log(y)
        
        # Rescale
        a = (band_1 - band_1.mean()) / (band_1.max() - band_1.min())
        b = (band_2 - band_2.mean()) / (band_2.max() - band_2.min())
        c = (band_3 - band_3.mean()) / (band_3.max() - band_3.min())

        imgs.append(np.dstack((a, b, c)))

    return np.array(imgs)

In [5]:
Xtrain = get_scaled_imgs(train)
Ytrain = np.array(train['is_iceberg'])

In [6]:
train.inc_angle = train.inc_angle.replace('na',0)
idx_tr = np.where(train.inc_angle>0)

In [7]:
len(Xtrain)

1604

In [8]:
Ytrain = Ytrain[idx_tr[0]]
Xtrain = Xtrain[idx_tr[0],...]

In [9]:
len(Xtrain)

1471

In [10]:
def get_more_images(imgs):
    
    more_images = []
    vert_flip_imgs = []
    hori_flip_imgs = []
      
    for i in range(0,imgs.shape[0]):
        a=imgs[i,:,:,0]
        b=imgs[i,:,:,1]
        c=imgs[i,:,:,2]
        
        av=cv2.flip(a,1)
        ah=cv2.flip(a,0)
        bv=cv2.flip(b,1)
        bh=cv2.flip(b,0)
        cv=cv2.flip(c,1)
        ch=cv2.flip(c,0)
        
        vert_flip_imgs.append(np.dstack((av, bv, cv)))
        hori_flip_imgs.append(np.dstack((ah, bh, ch)))
      
    v = np.array(vert_flip_imgs)
    h = np.array(hori_flip_imgs)
       
    more_images = np.concatenate((imgs,v,h))
    
    return more_images

In [11]:
Xtr_more = get_more_images(Xtrain) 
Ytr_more = np.concatenate((Ytrain,Ytrain,Ytrain))

In [26]:
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(128, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.3))

    #Conv Layer 2
    gmodel.add(Conv2D(256, kernel_size=(3, 3), activation='relu' ))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.3))

    #Conv Layer 3
    gmodel.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.3))

    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.4))

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.4))

    #Dense Layer 2
    gmodel.add(Dense(128))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.4))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.005, beta_1=0.9, beta_2=0.9, epsilon=1e-08, decay=0.01)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)

In [27]:
model = getModel()
model.summary()

batch_size = 16
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('.mdl_wts.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 73, 73, 128)       3584      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 36, 36, 128)       0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 36, 36, 128)       0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 34, 34, 256)       295168    
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 17, 17, 256)       0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 17, 17, 256)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 15, 15, 256)       590080    
__________

In [28]:
model.fit(Xtr_more, Ytr_more, batch_size=batch_size, epochs=30, verbose=1, callbacks=[earlyStopping, mcp_save, reduce_lr_loss], validation_split=0.2)

Train on 3530 samples, validate on 883 samples
Epoch 1/30

3530/3530 [==============================] - ETA: 0s - loss: 0.6532 - acc: 0.6499 - val_loss: 0.5526 - val_acc: 0.7418
Epoch 2/30

3530/3530 [==============================] - ETA: 0s - loss: 0.4750 - acc: 0.7652 - val_loss: 0.4932 - val_acc: 0.7327
Epoch 3/30

3530/3530 [==============================] - ETA: 0s - loss: 0.3887 - acc: 0.8150 - val_loss: 0.4482 - val_acc: 0.7984
Epoch 4/30

3530/3530 [==============================] - ETA: 0s - loss: 0.3607 - acc: 0.8278 - val_loss: 0.4391 - val_acc: 0.7928
Epoch 5/30

3530/3530 [==============================] - ETA: 0s - loss: 0.3280 - acc: 0.8484 - val_loss: 0.3423 - val_acc: 0.8256
Epoch 6/30

3530/3530 [==============================] - ETA: 0s - loss: 0.3143 - acc: 0.8558 - val_loss: 0.3385 - val_acc: 0.8301
Epoch 7/30

3530/3530 [==============================] - ETA: 0s - loss: 0.2941 - acc: 0.8660 - val_loss: 0.3888 - val_acc: 0.8211
Epoch 8/30

3530/3530 [=============

In [29]:
model.load_weights(filepath = '.mdl_wts.hdf5')

score = model.evaluate(Xtrain, Ytrain, verbose=1)
print('Train score:', score[0])
print('Train accuracy:', score[1])


1471/1471 [==============================] - ETA: 0s
Train score: 0.224613340482
Train accuracy: 0.916383412644


In [30]:
test = pd.read_json('test.json')
test.inc_angle = test.inc_angle.replace('na',0)
Xtest = (get_scaled_imgs(test))
pred_test = model.predict(Xtest)

submission = pd.DataFrame({'id': test["id"], 'is_iceberg': pred_test.reshape((pred_test.shape[0]))})
print(submission.head(10))

submission.to_csv('./submission/submission_2.csv', index=False)

         id  is_iceberg
0  5941774d    0.253535
1  4023181e    0.473720
2  b20200e4    0.304394
3  e7f018bb    0.995573
4  4371c8c3    0.906367
5  a8d9b1fd    0.938672
6  29e7727e    0.271278
7  92a51ffb    0.997947
8  c769ac97    0.000432
9  aee0547d    0.000006
